In [1]:
from time import perf_counter
import pickle

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

from soynlp.tokenizer import LTokenizer
from PyKomoran import Komoran, DEFAULT_MODEL

from models.modified_sample.gpt2 import GPT2ModifiedSampleForCausalLM
from models.modified_sample.gpt2 import (
    TOKENS_WITH_BRACE,
    TOKENS_WITH_BRACKET,
    TOKENS_WITH_CLOSING_BRACE,
    TOKENS_WITH_CLOSING_BRACKET,
    TOKENS_WITH_CLOSING_PARENTHESIS,
    TOKENS_WITH_COLON,
    TOKENS_WITH_PARENTHESIS,
    TOKENS_WITH_SEMICOLON,
)

In [2]:
# model_path = "lexiconium/kogpt-trinity"
# revision = "punct_wrapper-related_words-minevalloss"


# device = "cuda" if torch.cuda.is_available() else "cpu"
# tokenizer = AutoTokenizer.from_pretrained(model_path, revision=revision, use_auth_token=True)
# model = AutoModelForCausalLM.from_pretrained(model_path, revision=revision, use_auth_token=True).to(device)
# model.eval()

In [3]:
model_path = "/opt/ml/final-project-level3-nlp-08/outputs/checkpoint-450"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2ModifiedSampleForCausalLM.from_pretrained(model_path).to(device)
model.eval()

GPT2ModifiedSampleForCausalLM(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplac

In [4]:
komoran = Komoran(DEFAULT_MODEL["FULL"])

In [14]:
wrap = lambda token: [token]


@torch.no_grad()
def genreate_from_input(
    input_text: str, max_length: int = 64, top_k: int = 16, top_p: float = 0.8, temperature: float = 1.0
):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Check generation time
    t = perf_counter()

    output = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        # no_repeat_ngram_size=2,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        bad_words_ids=[
            *list(map(wrap, TOKENS_WITH_BRACE)),
            *list(map(wrap, TOKENS_WITH_BRACKET)),
            *list(map(wrap, TOKENS_WITH_CLOSING_BRACE)),
            *list(map(wrap, TOKENS_WITH_CLOSING_BRACKET)),
            *list(map(wrap, TOKENS_WITH_CLOSING_PARENTHESIS)),
            *list(map(wrap, TOKENS_WITH_COLON)),
            *list(map(wrap, TOKENS_WITH_PARENTHESIS)),
            *list(map(wrap, TOKENS_WITH_SEMICOLON)),
            *list(map(wrap, [0, 2, 3, 4, 5, 6, 7, 8])),
        ],
    )
    # output = model.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output[0])

    spent = perf_counter() - t

    return generated_text, spent


In [6]:
# with open("/opt/ml/data/namuwiki_filtered_cohesion.pickle", "rb") as f:
#     cohesions = pickle.load(f)
#     l_cohesions = {word: score[0] for word, score in cohesions.items()}
#     l_tokenizer = LTokenizer(l_cohesions)

In [7]:
komoran = Komoran(DEFAULT_MODEL["FULL"])

In [22]:
prompt = "흰 고양이가 사발 안에 앉아 있다."

# input_text = f"@{prompt}@<d>\n"

# words = l_tokenizer(prompt)
# input_text = f"@{', '.join(words)}@<d>\n"

morphes = komoran.get_morphes_by_tags(
    prompt, tag_list=["NNG", "NNP", "NNB", "NP", "NR", "VV", "VA", "VCP", "VCN", "MAG"]
)
input_text = f"@{prompt}@{', '.join(morphes)}@<d>\n"
# input_text = f"@{', '.join(morphes)}@<d>\n"

for top_p in [0.8]:
    for top_k in [16]:
        print(f"========\n{top_p=}\n{top_k=}\n========\n")
        for n in range(1, 6):
            generated_text, spent = genreate_from_input(
                input_text, max_length=64, top_k=top_k, top_p=top_p, temperature=1.0
            )
            print(f"[{n}]:\n{generated_text}\n")
            # print(f"\ntime spent: {spent:.2f} sec")

top_p=0.8
top_k=16

[1]:
@흰 고양이가 사발 안에 앉아 있다.@희, 고양이, 사발, 안, 앉@<d> 
고양이 한 마리가
사발에 엎드려 있다.
가만히 바라보고 있는데
그 모습이 어찌나 귀여운지
사람들이 다가온다.
모두들 고양이에게 눈길을 준다.
그러나
이내 그 녀석의 눈빛은

[2]:
@흰 고양이가 사발 안에 앉아 있다.@희, 고양이, 사발, 안, 앉@<d> 
구름 한 점 없는 하늘에
눈이 내리고
사내는 하얀 고양이처럼 엎드려 있다.
점점이 박힌 별들 사이로
한참을 서성이다
어느새 사내의 정수리에선
하

[3]:
@흰 고양이가 사발 안에 앉아 있다.@희, 고양이, 사발, 안, 앉@<d> 
어두운 밤
고양이는 혼자였다.
사발에 엎드려 있다.
가늘게 떨리는 털 사이로
눈이 빛나고 있었다.
빛나는 눈동자 속에 있는
아득한 허공에는 흰 고양이

[4]:
@흰 고양이가 사발 안에 앉아 있다.@희, 고양이, 사발, 안, 앉@<d> 
사발에 흰 고양이 한 마리
앉아 있다.
한참을 가만히 있더니
고개 돌리며 제 발로
걸어 들어온다.
그냥 가기가 아쉬워서
발을 툭툭 차보니,
녀

[5]:
@흰 고양이가 사발 안에 앉아 있다.@희, 고양이, 사발, 안, 앉@<d> 
고양이 한 마리
사막 위에 홀로 앉은 고양이.
그의 털이 흰 이유는
아무도 찾지 않는 곳에 있기 때문.
홀로 모래밭에 엎드려 있는
그는 누구인가,
세상을 버리고

